# Linear regression

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
from itertools import product

import numpy as np
import xarray as xr
import xarray.ufuncs as xf
import matplotlib.pyplot as plt
from scipy.optimize import lsq_linear
from tqdm.auto import trange, tqdm

from system_identification.lsqr_model import LeastSquaresModel

In [3]:
data = xr.open_dataset("data_smoothed.nc")
data

<xarray.Dataset>
Dimensions:                 (t: 10001, dim_0: 4, dim_1: 4, x_idx: 4, training_samples: 8000, inputs: 2, m: 1, outputs: 1, validation_samples: 2001, d0: 2, d1: 2)
Coordinates:
  * t                       (t) float64 0.0 0.01 0.02 0.03 ... 99.98 99.99 100.0
Dimensions without coordinates: dim_0, dim_1, x_idx, training_samples, inputs, m, outputs, validation_samples, d0, d1
Data variables: (12/40)
    p_k1k1                  (t, dim_0, dim_1) float64 ...
    p_kk1                   (t, dim_0, dim_1) float64 ...
    phi                     (t, dim_0, dim_1) float64 ...
    gamma                   (t, dim_0, dim_1) float64 ...
    ps                      (t, dim_0, dim_1) float64 ...
    x_k1k1                  (t, x_idx) float64 ...
    ...                      ...
    alpha_estimate          (t) float64 ...
    training_inputs         (training_samples, inputs, m) float64 ...
    training_outputs        (training_samples, outputs, m) float64 ...
    validation_inputs       (validation_samples, inputs, m) float64 ...
    validation_outputs      (validation_samples, outputs, m) float64 ...
    input_range             (d0, d1) float64 ...

In [16]:
orders = list(range(1, 50, 5))

error_training_mean = []
error_training_abs_mean = []
error_training_jb = []
error_training_jbp = []
error_validation_mean = []
error_validation_abs_mean = []
error_validation_jb = []
error_validation_jbp = []

for order in tqdm(orders):
    model = LeastSquaresModel.new_polynomial(
        n_inputs=2,
        input_range=data.input_range.values,
        order=order)
    model

    model.train(
        inputs=data.training_inputs.values,
        reference_outputs=data.training_outputs.values,
        validation_inputs=data.validation_inputs.values,
        validation_outputs=data.validation_outputs.values,
    )
    
    error_training_mean.append(abs(model.training_log.error_training_data).mean())
    error_training_abs_mean.append(model.training_log.error_training_data.mean())
    error_training_jb.append(model.training_log.error_training_jb.item())
    error_training_jbp.append(model.training_log.error_training_jbp.item())
    
    error_validation_mean.append(abs(model.training_log.error_validation_data).mean())
    error_validation_abs_mean.append(model.training_log.error_validation_data.mean())
    error_validation_jb.append(model.training_log.error_validation_jb.item())
    error_validation_jbp.append(model.training_log.error_validation_jbp.item())
    
    

  0%|          | 0/10 [00:00<?, ?it/s]

In [18]:
fig, axs = plt.subplots(4, 1, figsize=(8, 8))

axs[0].set_yscale("log")
axs[0].plot(orders, error_training_mean)
axs[0].plot(orders, error_validation_mean)

axs[1].plot(orders, error_training_abs_mean)
axs[1].plot(orders, error_validation_abs_mean)

axs[2].set_yscale("log")
axs[2].plot(orders, error_training_jb)
axs[2].plot(orders, error_validation_jb)

axs[3].plot(orders, error_training_jbp)
axs[3].plot(orders, error_validation_jbp)

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
fig = plt.figure()

reference_inputs = data.evaluation_inputs.values
reference_outputs = data.evaluation_outputs.values

output = model.evaluate(reference_inputs)
ax = fig.add_subplot(projection='3d')
ax.scatter(reference_inputs[:, 0, 0], reference_inputs[:, 1, 0], reference_outputs.squeeze(), s=0.1)
ax.scatter(reference_inputs[:, 0, 0], reference_inputs[:, 1, 0], output.squeeze(), s=0.1)

In [ ]:
model.training_log